# ART tutorial for MNIST dataset

Welcome to the **ART** tutorial for the MNIST dataset. In this tutorial, we will demonstrate how to perform a deep learning experiment using ART.

The tutorial is divided into the following sections:
1. Introduction to ART
2. Installation of required packages
3. Basic usage of ART 

## 1. Introduction to ART

ART is designed to support deep learning researchers providing a special pipeline to perform all experments in a unified way. Each experiment is defined of the following components:
- **Dataset**: Each Project is supposed to deal with one dataset. ART supports every LightningDataModule from PyTorch Lightning, and every torch Dataset with its own dataloader.
- **Model**: While exploring different models, ART provides a unified way to define and train models. ART uses **ArtModule** that inherits from PyTorch Lightning's LightningModule. ArtModules are designed to be easily configurable and to support different model architectures.
- **Step**: Unitary process that takes you closer to your final goal - good Deep Learning model. **In this tutorial we present you steps that were inspired by Andrej Karpathy's** [Recipe for Training Neural Networks](http://karpathy.github.io/2019/04/25/recipe/):
    1. EvaluateBaseline - before starting a new project it is good to know with whom we compete.
    2. CheckLossOnInit - Checking Loss right after network initialization is a very good debug step
    3. OverfitOneBatch - If you can't Overfit single batch, it is very unlikely that your network will work any good.
    4. Overfit - By reducing wanted metric on training set you can observe `theoreticall` achievable minimum if this value doesn't satisfy you it is very unlikely it will be better on the test set.
    5. Regularize - Usually gap between training and validation score is quite big and we need to introduce regularization techniques to achieve satisfactory validation accuracy.
- **Metrics**: To make you write less code we implemented `MetricCalculator`, a special object that takes care of metric calculation between all steps. The only thing you have to do is to `register` metrics that you want to compute.
- **Check**: For every step you must pass a list of `Check` objects that must be fulfilled for the step to be passed. You may encounter checks like `CheckScoreExists` or `CheckCloseTo`. Every check takes at least 3 arguments:
  - Metric, which value at the end of a step will be checked. This may be e.g `nn.CrossEntropyLoss` object.
  - Stage, During different steps you'll be interested in performance at either Training or Validation. You must pass information about this too.
  - value, wanted value of the check
- **Art Project**: Every project consists of many steps. You can add them to `ArtProject` which is responsible for running them. `ArtProject` also saves metadata about your steps that you can later see in `Dasboard`. 



## 2. Install dependencies

At this stage we assume that you have already installed the ART package. If not, please refer to the [ART README](https://github.com/SebChw/art#readme)

We will also need to install the following packages:

In [ ]:
!pip install transformers albumentations torchvision einops lovely-tensors torchmetrics

## 3. Basic usage of ART
Normally after cloning the project template you would have to implement every code template inside `exp1` on your own. We would just provide the project skeleton with all classes and functions defined, but not implemented. For this tutorial, we implemented all the functions so that you can observe how to use art.

Let's tackle this strong enemy with the art framework.

### Data preparation
First, we need to download the data. We highly recommend using Huggingface datasets wrapped into the Lightning Data Module. You can also use Torch Datasets and DataLoaders. This time we prepared `MNISTDataModoule` for you. But if you work with your custom data you must implement it by yourself. Look inside `art_data.py` to see how we've done this


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lightning import seed_everything
seed_everything(23)

In [ ]:
import datasets
from art_data import MNISTDataModule
from art.utils.enums import INPUT, TARGET

def get_data_module(n_train=200):
    mnist_data = datasets.load_dataset("mnist")

    mnist_data = mnist_data.rename_columns({"image": INPUT, "label": TARGET})
    mnist_data['train'] = mnist_data['train'].select(range(n_train))

    return MNISTDataModule(mnist_data)


mnist_data_module = get_data_module()

### Data analysis

In every project we encourage you to start with a deep look into the data. Currently, we are working on a tool that will help you with this task. For now, we left this part for you to implement on your own.

In [ ]:
from steps import DataAnalysis
from art.experiment.Experiment import ArtProject
from art.step.checks import CheckResultExists
from checks import CheckClassImagesExist, CheckLenClassNamesEqualToNumClasses

project = ArtProject("exp1", mnist_data_module)
project.add_step(DataAnalysis(), [
    CheckResultExists("number_of_classes"),
    CheckResultExists("class_names"),
    CheckResultExists("number_of_examples_in_each_class"),
    CheckResultExists("img_dimensions"),
    CheckClassImagesExist(),
    CheckLenClassNamesEqualToNumClasses()])

In [ ]:
project.run_all()

### Metrics & experiment definition

1. We define the **Experiment** - the heart of art. Later we will add steps that will be performed for this particular experiment.
2. We register some metrics in **MetricCalculator** - this class performs metric calculations for all models defined by you during every step. The only need you must provide is a consistent format of returned data from different models. You can easily add exceptions - we don't want to calculate the loss for the Evaluate Baseline Step. If some metrics are expensive to calculate you add an exception and it won't be calculated. You don't need to modify the code to modify metrics behavior.

In [ ]:
from torchmetrics import Accuracy
import torch.nn as nn
from art.step.steps import EvaluateBaseline

NUM_CLASSES = project.get_step(0).get_latest_run()["number_of_classes"]
accuracy_metric, ce_loss = Accuracy(task="multiclass", num_classes = NUM_CLASSES), nn.CrossEntropyLoss()
project.register_metrics([accuracy_metric, ce_loss])

### Baselines

Starting any Machine Learning project it is good to have some baseline for comparison. Here in `ART` we try to encourage you to try at least one of those:
* ML Baseline - Not for all problems we need Deep Learning. For tabular data or time series, there are ML models that you can take off the shelf and obtain very good results. 
* Heuristic Baseline - Sometimes you may have some intuition on how should be data regressed or classified. It is always worth to check how good these are.
* Existing solution baseline - If the problem is already solved, checking existing solutions and pointing out their drawbacks to improve on is always worth doing.


Another good habit is to separate your code into many smaller modules and don't do everything inside one gigantic Jupyter notebook. Look inside the `baselines.py` file where we have prepared everything for you.

Have you observed plenty of small functions inside each of the classes? These will create pipelines of your ArtModule.

E.g. `MlBaseline` has functions `parse_data_for_ml` and `baseline_train` that both create `ml_train_pipeline` that is defined inside art. As you become more familiar with `ART` you will be able to create your own pipelines and their building blocks.

Finally, to offer you a high level of flexibility we pass data between all stages with the use of a dictionary (This will be changed into a better structure in the future).


In [ ]:
from art.core.MetricCalculator import SkippedMetric
from baselines import MlBaseline, HeuristicBaseline, AlreadyExistingSolutionBaseline
from art.step.checks import CheckScoreExists
from art.utils.enums import TrainingStage
baselines = [HeuristicBaseline, MlBaseline, AlreadyExistingSolutionBaseline]
for baseline in baselines:
    project.add_step(
        step = EvaluateBaseline(baseline), 
        checks = [CheckScoreExists(metric=accuracy_metric)],
        skipped_metrics=[SkippedMetric(metric=ce_loss)]
    )

Here we defined our 3 baselines. You can check how these are structured in `baselines.py` file. Next we create our first step that will consist of their evaluation. We always add new steps in a form: `step to be performd`, `list_of_check_to_be_performed`

### First run of the experiment

In [ ]:
project.run_all()

Alright, a lot has happened. But in the end, we have validated three baselines successfully. We can see, that the already existing solution is a very strong competitor. If results obtained by these baselines are satisfactory. 

Please visit the checkpoints folder - results of this stage are saved here. `In version 2.0 you will be able to see all the results from the dashboard`.

Having baselines we can move on and try to beat them. We will use CNN for this purpose which is defined inside `models.py`. The next step is to validate loss after network initialization. If it is off, we probably messed something up in our pipeline. We have 10 classes, so on average with random initialization loss should be -log(1/10) = 2.31


### Adding CheckLossOnInitStep

In [ ]:
import math
EXPECTED_LOSS = - math.log(1 / NUM_CLASSES)
EXPECTED_LOSS

In [ ]:
from models import MNISTModel
from art.step.steps import CheckLossOnInit
from art.step.checks import CheckScoreCloseTo
from torch.cuda import is_available
project.to("cuda" if is_available() else "cpu")

project.add_step(
    CheckLossOnInit(MNISTModel),
    [CheckScoreCloseTo(metric=ce_loss, value=EXPECTED_LOSS, rel_tol=0.1)]) 
project.run_all()

And here we are, something must be wrong in our pipeline - loss is off by big amount. Observe, two things:
1. We didn't perform baselines evaluations again this step is already passed
2. We added a check to our step. We want our loss to be close to 2.3 we failed to provide this result so we obtained an exception. We can't go further until we pass this stage.

Let's discover another feature of art that helps you to debug the code and find problems with your pipeline - Decorators.

### Decorators

`TODO`: make an introduction what is `set visualization`

In [ ]:
import lovely_tensors as lt
from art.decorators import art_decorate
import logging
lt.monkey_patch()

logging.basicConfig(filename='debug_art.log', level=logging.WARNING)
def first_logging_func(data):
    logging.warning(data)


art_decorate([(MNISTModel, "predict")], first_logging_func)
project.run_all()

You may decorate any function in your pipeline with any logging function. Here it is a simple log, but in practice, you can do much, much more, which you will see later. Please visit the `debug_art.log` file. 

Can you spot what is wrong? Yes! We do not normalize images to the [0,1] range.

In [ ]:
from models import MNISTModelNormalized
project.replace_step(CheckLossOnInit(MNISTModelNormalized))
project.run_all()

And we've done it! We gained a little more confidence in our pipeline. Let's move further. The next step is to overfit just a single batch. If you can't do this there is for sure something wrong. From now procedure for everything else is very similar. So we import the next step and add it to our experiment with the appropriate check.

### Another step - overfitting one batch

In [ ]:
from art.step.steps import OverfitOneBatch
from art.step.checks import CheckScoreLessThan

project.add_step(OverfitOneBatch(MNISTModelNormalized),
            [CheckScoreLessThan(metric=ce_loss, value=0.001)])
project.run_all()

Oh no... we can't overfit. Is this an issue with the model? Maybe but at first let's see another powerful use case of art. We will debug the prediction dynamics to see how our model is evolving. We will calculate the average logit value for all images that should be classified as 1. For this purpose we create a class with memory and make it callable, later we decorate our model prediction function and casually run the experiment.

In [ ]:
class EvolutionSaver:
    def __init__(self):
        self.logits = []
        self.time = 0

    def __call__(self, data):
        targets = data['target'] == 1
        logits = data['prediction']
        
        wanted_logits = logits[targets].mean(dim=0)
        for i, logit in enumerate(wanted_logits):
            self.logits.append({
                "time": self.time,
                "logit": logit.item(),
                "class": i
            })

        self.time += 1    

    def visualize(self):
        import pandas as pd
        if len(self.logits) == 0:
            print("Step was not run and logits are empty")
            return

        df = pd.DataFrame(self.logits)
        df = df.pivot(index='time', columns='class', values='logit')
        self.logits = []
        self.time = 0
        return df.plot(xlabel="epoch number", ylabel="logit value", title="Evolution of logits values when digit 1 is a target")

evolution_saver = EvolutionSaver()
art_decorate([(MNISTModelNormalized, "predict")], None, evolution_saver)
project.replace_step(OverfitOneBatch(MNISTModelNormalized))
project.run_all()

The experiment fails, we know that. But what about the visualization?

In [ ]:
plot = evolution_saver.visualize()

But how about saving this plot to a logging service like Neptune?

Here comes yet another help from art called logger adapter.
Thanks to this, you can always get back to the visualization you have created at any time - even if you use an external computing unit.

Create a Neptune project, paste your api_token and project name, and use the logger as easy as it is.

In [ ]:
NEPTUNE_LOGGING = False # set to True if you want to log to neptune.ai

In [ ]:
if NEPTUNE_LOGGING:
    from art.utils.LoggerAdapter import NeptuneLoggerAdapter
    from neptune.exceptions import NeptuneMissingApiTokenException
    project_name = input("Provide neptune project name")
    experiment_name = "art-overfitting-one-batch"
    try:
        logger = NeptuneLoggerAdapter(project=project_name, custom_run_id=experiment_name)
        if plot is not None:
            logger.log_figure(plot.get_figure(), "logit_evolution")
    except NeptuneMissingApiTokenException as e:
        print("To enable logging, please set your NEPTUNE_API_TOKEN environment variable and reset the notebook, or pass your token and project name directly to NeptuneLoggerAdapter like:\n    NeptuneLoggerAdapter(api_token=your_token, project=project_name)\n")
        print("For more information, please visit https://docs.neptune.ai/setup/setting_api_token/.")
else:
    logger = None

Let's get back to debugging.

So, as you can see we are moving in the correct direction. 1 has the biggest logist very soon but the difference is not big enough yet - Probably we should just wait a little bit longer. Moreover, we see how powerful decorators can be. This helps us make our code more modular and fulfill the Open-Closed principle.

We can solve our problem in two ways:
* Either increase the learning rate.
* Or train for longer.

In [ ]:
project.replace_step(OverfitOneBatch(MNISTModelNormalized, number_of_steps=200))
project.run_all()
plot = evolution_saver.visualize()
if plot is not None and logger is not None:
    logger.log_figure(plot.get_figure(), "logit_evolution2")

In [ ]:
if NEPTUNE_LOGGING: logger.stop()

Nice! At overfitting process we'd recommend not to play with learning rate and stick with default value. It is better to just wait for a little bit longer. Now another Step can be taken. Overfitting entire training set. This is also very important as this somehow approximates our best possible score that can be achieved. If it is much below our expected result this again may indicate problems.

### Overfitting entire training set

In [ ]:
from art.step.steps import Overfit
from art.step.checks import CheckScoreGreaterThan
if NEPTUNE_LOGGING:
    experiment_name = "overfitting-train-set"
    logger = NeptuneLoggerAdapter(project=project_name, custom_run_id=experiment_name)

project.add_step(Overfit(MNISTModelNormalized, max_epochs=10, logger=logger), 
            [CheckScoreGreaterThan(metric=accuracy_metric, value=0.8)])

In [ ]:
project.run_all()

In [ ]:
if NEPTUNE_LOGGING: logger.stop()

Nice, We were able to pass the next stage in the first try. Hopefully you see another challenge that we have to face. We are able to obtain nice accuracy on the training stage. But on the validation it decreases by very much. We are overfitting the training set. For this tutorial training set was chosen to be very small so this problem was expected. But now let's try to pass the finals step of this tutorial, namely regularization. When regularizing the model you should start from the most powerful regularization technique and go to weaker. Here we will use:

1. getting more data - definitely one of the most powerful techniques. If possible always prefer this one.
2. augmentations - also very strong one

When applying augmentations to the images it is crucial to verify how do they influence the image. We again can use our decoration possibilities. *Under the hood `Regularize` class runs turn_on_regularization of ArtModule and the DataModule you use*.

You can see what we input to the network by checking `augmentations_i.png` files.

In [ ]:
from torchvision.utils import save_image
from art.step.steps import Regularize

class BatchSaver:
    def __init__(self):
        self.time = 0

    def __call__(self, data):
        if self.time < 10:
            save_image(data['input'], f'augmentations_{self.time}.png')
        self.time +=1
if NEPTUNE_LOGGING:
    experiment_name = "regularization"
    logger = NeptuneLoggerAdapter(project=project_name, custom_run_id=experiment_name)

mnist_data_module = get_data_module(n_train=2000)
project.update_datamodule(mnist_data_module)
art_decorate([(MNISTModelNormalized, "predict")], BatchSaver())
project.add_step(Regularize(MNISTModelNormalized, logger=None, trainer_kwargs={"max_epochs":10, "check_val_every_n_epoch":10}), 
      [CheckScoreGreaterThan(metric=accuracy_metric, value=0.80)])

project.run_all()

In [ ]:
if NEPTUNE_LOGGING: logger.stop()

Here you are! Hopefully, now you have incentive to try ART for your next data science project.
